In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/transact_18_22/transact_18_22.csv")
start_date = '2019-01-015'
end_date = '2019-12-14'

mask = (df['date'] >= start_date) & (df['date'] <= end_date)
df = df.loc[mask]
df

In [ ]:
prev_categories_to_mcc = df[["mcc", "group"]].groupby("group").agg(set).tail(14).reset_index()
prev_categories_to_mcc

In [ ]:
sssr = {
    "food": "survival",
    "clothes_and_outfit": "survival",
    "dwelling_and_home_services": "survival",
    "healthcare_and_medicine": "survival",

    "remote_trade": "socialization",
    "travel_and_public_transport": "socialization",
    "non_food_goods": "socialization",
    "telecommunications": "socialization",
    "miscellaneous_expenses": "socialization",

    "beauty_and_cosmetics": "self_realization",
    # "kids": "self_realization",
    "fun_and_entertainment": "self_realization",
    "charity": "self_realization",

    "money": "money"
}

In [ ]:
categories_to_mcc = {
    'survival': [
        5411, 5422, 5441, 5451, 5462, 5499, 5811, 5812, 5814, 5921, 5611, 5621, 5631, 5641, 5651, 5655, 5661, 5681, 5691, 5697, 5698, 5699,
        1520, 1711, 1731, 1740, 1750, 1761, 1771, 1799, 5211, 5231, 5251, 5261, 5271, 7011, 742, 5912, 5975, 5976, 7210, 7211, 7216, 7217,
        7261, 7297, 7298, 8011, 8021, 8031, 8041, 8042, 8043, 8049, 8050, 8062, 8071, 8099, 4900, 763, 780, 5541, 5542, *range(5712, 5723)
    ],

    'socialization': [
        *range(3000, 3351), *range(3351, 3501), *range(3501, 4000),
        4011, 4111, 4112, 4119, 4121, 4131, 4214, 4215, 4225, 4411, 4457, 4468, 4511, 4582, 4722, 4784, 4789, 7012, 7032, 7033, 7512, 7513,
        7519, 7523, 4812, 4813, 4814, 4816, 4821, 4899, 5200, 5300, 5309, 5310, 5311, 5331, 5399, 7273, 7276, 7277, 7278, 7296, 8111
    ],

    'self_realization': [
        5732, 5733, 5734, 5735, 5813, 5815, 5816, 5817, 5818, 5941, 5942, 5945, 5946, 5947, 5970, 5971, 5972, 5973, 7221, 7230, 7251, 7829,
        7832, 7841, 7911, 7922, 7929, 7932, 7933, 7941, 7991, 7992, 7993, 7994, 7996, 7997, 7998, 7999, 5943, 8211, 8220, 8241, 8244, 8249,
        8299, 8351, 5977, 5978, 5997, 8398, 5551, 5561
    ],

    'business': [
        2741, 2791, 2842, 5013, 5021, 5039, 5044, 5045, 5046, 5047, 5051, 5065, 5072, 5074, 5085, 5094, 5099, 5111, 5122, 5131, 5137, 5139,
        5169, 5172, 5192, 5193, 5198, 5199, 7311, 7321, 7322, 7333, 7338, 7339, 7342, 7349, 7361, 7372, 7375, 7379, 7392, 7393, 7394, 7395,
        7399, 8911, 8931, 8999, 5531, 5532, 5533, 7531, 7534, 7535, 7538, 7542, 7549, 5511, 5521, 5571, 5599
    ],

    'government': [
        9211, 9222, 9223, 9311, 9399, 9402, 9405, 9406, 8641, 8651, 8661, 8675, 8699, 8734
    ],

    'money': [
        4829, 6010, 6011, 6012, 6050, 6051, 6211, 6300, 6513, 6532, 6533, 6536, 6537, 6538, 6540
    ],

    'special': [
        7800, 7801, 7802, 7995, 5931, 5932, 5933, 5935, 5937, 5940, 5944, 5948, 5949, 5950, 5960, 5962, 5963, 5964, 5965, 5966, 5967, 5968,
        5969, 5983, 5992, 5993, 5994, 5995, 5996, 5998, 5999, 7295, 7299, 7622, 7623, 7629, 7631, 7641, 7692, 7699
    ]
}

In [ ]:
categorized_mccs = set()
for k in categories_to_mcc:
    categorized_mccs |= set(categories_to_mcc[k])

len(categorized_mccs)

In [ ]:
def get_category(mcc):
    categories = []
    for category in categories_to_mcc:
        if mcc in categories_to_mcc[category]:
            categories.append(category)

    if len(categories) == 0 or len(categories) > 1:
        print(f"More than one or zero category for MCC, mcc_code={mcc}, categories={categories}")
    return categories[0]

def get_sssr(mcc):
    category = get_category(mcc)
    return sssr[category]

In [ ]:
set(df["mcc"].unique().tolist()).difference(categorized_mccs)

In [ ]:
prepared_df = df[["client", "card", "date", "amt", "mcc"]].copy()
prepared_df.loc[:, "group"] = prepared_df["mcc"].apply(get_category)
prepared_df.drop(["mcc", "card"], axis=1, inplace=True)

In [ ]:
prepared_df = prepared_df[(prepared_df["group"] == "survival") | (prepared_df["group"] == "socialization") | (prepared_df["group"] == "self_realization")]

In [ ]:
prepared_df = prepared_df.groupby(["client", "date"]).agg(list)
prepared_df

In [ ]:
basic_value = 200
categories = {"survival": 0, "socialization": 1, "self_realization": 2}

cnt = 0
for id, row in prepared_df.iterrows():
    cnt += 1
    if cnt % 100000 == 0:
        print(cnt)

    amounts = row["amt"]
    groups = row["group"]

    res = [0, 0, 0]
    for value, category in zip(amounts, groups):
        if value >= basic_value:
            res[categories[category]] = 1

    prepared_df.loc[id, "survival"] = res[0]
    prepared_df.loc[id, "socialization"] = res[1]
    prepared_df.loc[id, "self_realization"] = res[2]
    prepared_df.loc[id, "code"] = res[0] + res[1] * 2 + res[2] * 4



In [ ]:
prepared_df

In [ ]:
prepared_df = prepared_df[["survival", "socialization", "self_realization", "code"]]
prepared_df

In [ ]:
prepared_df.to_csv("processed/transact_18_22.csv", index=True)